In [7]:
import requests as r
import bs4
import urllib
from newspaper import Article
import spacy
import re
import pandas as pd

In [8]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /home/novasangeeth/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [9]:

# !python -m spacy download en_core_web_sm


In [10]:
nlp = spacy.load("en_core_web_sm")

In [11]:
url_main = "https://www.google.com/search"
query = "RCB"
tbm = "nws"
start = "15"

URL = f"{url_main}?q={query}&tbm={tbm}&start={start}"


In [12]:
url = "https://economictimes.indiatimes.com/small-biz/sme-sector/5-ways-your-company-can-be-carbon-neutral/articleshow/91431060.cms"


def get_html(url):
    response = r.get(url)
    return response.content


def get_content(html):
    soup = bs4.BeautifulSoup(html, 'html.parser')
    return soup.find_all('p')


def get_text(content) -> list:
    text = []
    for i in content:
        text.append(i.text)
    return text


def check_url_validity(urls) -> bool:
    for i in urls:
        req = r.get(i)
        return True if req.status_code == 200 else False


def clean_urls(urls) -> list:
    cleaned_urls = []
    for i in urls:
        if i.startswith('/url?q=https://www'):
            link = i.replace('/url?q=', '')
            cleaned_urls.append(link)
    return cleaned_urls


def get_hrefs(html) -> list:
    soup = bs4.BeautifulSoup(html, 'html.parser')
    hrefs = []
    for i in soup.find_all('a'):
        hrefs.append(i.get('href'))
    return hrefs


def get_article(url):
    article = Article(url)
    article.download()
    article.parse()
    return article.text


def list_to_string(list) -> str:
    return ' '.join(list)


def remove_special_characters(text) -> str:
    cleaned_text = re.sub(r"[^a-zA-Z0-9 ]", "", text)
    return cleaned_text


def get_keywords(text, count=None) -> list:
    doc = nlp(text)
    words = []
    for token in doc:
        if token.is_stop != True:
            words.append(token.text)

    keywords = list(set(words))
    return len(keywords) if count else keywords

def get_domain_name(url) -> str:
    return urllib.parse.urlparse(url).netloc






In [13]:

html = get_html(url)
content = get_content(html)
text = list_to_string(get_text(content))
cleaned_text = remove_special_characters(text)
keywords = get_keywords(cleaned_text)
keywords

['Articles',
 'Categories',
 'Calculators',
 'Prime',
 'investing',
 'Topics',
 'Searched',
 'Stories',
 'moderators',
 'displayed',
 'Economic',
 'Aswath',
 'Updates',
 'guru',
 'inflationary',
 'News',
 'button',
 'action',
 'highs',
 'better',
 'Slideshow',
 'Damodarans',
 'Report',
 'interested',
 'storm',
 'Web',
 'reason',
 'InConnect',
 'Log',
 'exactly',
 'Market',
 'digital',
 'App',
 'missed',
 'reading',
 'experience',
 'weather',
 'stories',
 'Sensex',
 'likely',
 'alert',
 'Daily',
 'Indulge',
 'scrip',
 'Stock',
 'Live',
 'reporting',
 'Story',
 'Find',
 'offensive',
 'retest',
 'tips',
 'amid',
 'genie',
 'comment',
 'Reported',
 'Case',
 '2021',
 'Hot',
 'Videos',
 'Companies',
 'Popular',
 'ETPrime',
 'admin',
 'Choose',
 'day',
 'target',
 'Rs',
 'newspaper',
 'November',
 'click',
 'buyondips',
 'ET',
 'Listing',
 'Definitions',
 'Download',
 'Finance',
 'companies',
 'inflation',
 'times',
 '870',
 'Business',
 'Radar',
 'Commodities',
 'Times',
 'Reason',
 ' ',
 'P

In [16]:
def scrape_rss():
    # url = "https://www.thehindu.com/news/national/tamil-nadu/feeder/default.rss"
    url = "https://www.bing.com/news/search?q=singapore&format=rss"
    # url = "https://timesofindia.indiatimes.com/rssfeeds/296589292.cms"
    # url = "https://www.bing.com/news/search?q=india&format=rss&count=10&offset=0"
    response = r.get(url)
    soup = bs4.BeautifulSoup(response.content, 'xml')
    news = soup.find_all('item')
    # print(news)
    result = []
    for i in news:
        title = i.title.text
        category = i.category.text
        link = i.link.text
        description = i.description.text
        pub_date = i.pubDate.text
        article_content = get_article(link)
        item = {
            'title': title.strip(),
            'category': category.strip(),
            'link': link.strip(),
            'description': description.strip(),
            'pubDate': pub_date.strip(),
            # 'article_content': article_content.strip()
        }
        result.append(item)
    return result

# scrape_rss()


In [17]:
rss_df = pd.DataFrame(scrape_rss())

AttributeError: 'NoneType' object has no attribute 'text'

In [ ]:
rss_df

,title,category,link,description,pubDate
0,T.N. moves SC against CWMA taking up Mekedatu ...,Tamil Nadu,https://www.thehindu.com/news/national/tamil-n...,State representative will point out that this ...,"Tue, 07 Jun 2022 20:20:39 +0530"
1,MRB nurses protest demanding regularisation of...,Tamil Nadu,https://www.thehindu.com/news/national/tamil-n...,"Over 2,600 nurses take part in the State-wide ...","Tue, 07 Jun 2022 19:34:47 +0530"
2,YouTuber Karthik Gopinath moves HC to quash FIR,Tamil Nadu,https://www.thehindu.com/news/national/tamil-n...,Avadi police too approach the HC seeking permi...,"Tue, 07 Jun 2022 19:22:14 +0530"
3,No firm blacklisted after row over Pongal gift...,Tamil Nadu,https://www.thehindu.com/news/national/tamil-n...,"It says ₹1,163 crore had been set apart for th...","Tue, 07 Jun 2022 19:07:42 +0530"
4,Act on petitions from the general public swift...,Tamil Nadu,https://www.thehindu.com/news/national/tamil-n...,,"Tue, 07 Jun 2022 18:40:27 +0530"
5,Panneerselvam writes to Modi seeking NEET exem...,Tamil Nadu,https://www.thehindu.com/news/national/tamil-n...,"They should be regarded as a special case, say...","Tue, 07 Jun 2022 16:29:44 +0530"
6,Rear Admiral calls on CM,Tamil Nadu,https://www.thehindu.com/news/national/tamil-n...,,"Tue, 07 Jun 2022 18:45:12 +0530"
7,HR and CE team inspects Sri Sabanayagar temple...,Tamil Nadu,https://www.thehindu.com/news/national/tamil-n...,They say authorities have no power to call for...,"Tue, 07 Jun 2022 15:15:52 +0530"
8,Entry of tourists banned in Hogenakkal as Cauv...,Tamil Nadu,https://www.thehindu.com/news/national/tamil-n...,Rain in Karnataka cited as the reason for the ...,"Tue, 07 Jun 2022 14:13:06 +0530"
9,Stalin launches training programmes for fisher...,Tamil Nadu,https://www.thehindu.com/news/national/tamil-n...,It is aimed at helping them learn life-saving ...,"Tue, 07 Jun 2022 13:16:52 +0530"


In [ ]:
url = "https://www.bing.com/news/search?q=india&format=rss&count=10&offset=0"
html = get_html(url)
content = get_content(html)

soup = bs4.BeautifulSoup(html, 'html.parser')

soup.find_all("a", class_="title")


[]

In [ ]:

"""
Kishore S8:47 PM
https://www.bing.com/news/search?q=india&format=rss&count=30&offset=0
https://stackoverflow.com/questions/61078960/bing-search-api-v7-pagination
https://docs.microsoft.com/en-us/rest/api/cognitiveservices-bingsearch/bing-news-api-v7-reference
"""


In [ ]:
soup.find_all('item')

In [ ]:
# function to save the dataframe into a csv or json or excel based based on the argument passed in.
def save_data(
    df,
    file_name,
    file_path,
    file_type
):
    if not file_path:
        file_path = '.'

    full_file_name = f"{file_path}/{file_name}.{file_type}"
    if file_type == 'csv':
        df.to_csv(full_file_name, index=False)
    elif file_type == 'json':
        df.to_json(full_file_name, orient='records')
    elif file_type == 'excel':
        df.to_excel(full_file_name, index=False)
    else:
        print('Invalid file type')
